<a href="https://colab.research.google.com/github/dabin3178/sshs_230906/blob/main/(230906_%EC%84%9C%EC%9A%B8%EA%B3%BC%EA%B3%A0)KoBERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
# 설치 후 런타임 다시 시작 필요할 수도 (런타임 다시 시작 필요하면 실행창에 뜸)
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292696 sha256=896b02396a458bad1d92cb66386e02e4124dfa5edfbca33b788497fca9dc745a
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-k9thtm9v/kobert-tokenizer_993bb0725eee48869abaf209c6d835a4
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-k9thtm9v/kobert-tokenizer_993bb0725eee48869abaf209c6d835a4
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=9bb76a6e61a501495dac6c32ecb2f7d1ce1b05182d0086a6c988a2801d57238c
  Stored in directory: /tmp/pip-ephem-wheel-cache-07boufth/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
!pip3 install kobert-transformers

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 파라미터 설정
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
data_pd = pd.read_csv('/content/drive/MyDrive/LAMDA/mutil_labeled_everytime_data(merged).csv')
data_pd

,Unnamed: 0,게시글,1,2,3,4,5,6,7,8,9
0,0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,모든 게 후회된다,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,임신하고 나서 우울하고 의욕이 없어,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,패배감 너무 심해서 어디 상담이라도 받고 싶어,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
59908,59908,그래도 잠못자고 불안한건 여전해요.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59909,59909,불안함에 항상 시달리니까 잠도 못잤어요.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59910,59910,불안하고 초조해서 잠이 안 와.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59911,59911,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_pd = data_pd.drop(data_pd.columns[[0]],axis=1)
data_pd

,게시글,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,모든 게 후회된다,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,임신하고 나서 우울하고 의욕이 없어,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,패배감 너무 심해서 어디 상담이라도 받고 싶어,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
59908,그래도 잠못자고 불안한건 여전해요.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59909,불안함에 항상 시달리니까 잠도 못잤어요.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59910,불안하고 초조해서 잠이 안 와.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59911,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 굳이 안 필요한 과정일 수도?
data_pd = data_pd.astype({'1':int, '2':int, '3':int, '4':int, '5':int, '6':int, '7':int, '8':int, '9':int})
data_pd

,게시글,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,1,0,0,0,0,0,0,0,0
1,모든 게 후회된다,1,0,0,0,0,0,0,0,0
2,임신하고 나서 우울하고 의욕이 없어,1,0,0,0,0,0,0,0,0
3,패배감 너무 심해서 어디 상담이라도 받고 싶어,1,1,0,0,0,0,0,0,0
4,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
59908,그래도 잠못자고 불안한건 여전해요.,0,0,0,0,0,0,0,0,0
59909,불안함에 항상 시달리니까 잠도 못잤어요.,0,0,0,0,0,0,0,0,0
59910,불안하고 초조해서 잠이 안 와.,0,0,0,0,0,0,0,0,0
59911,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,0,0,0,0,0,0,0,0,0


In [ ]:
for column in data_pd.columns[1:]:
    data_pd[column] = data_pd[column].apply(lambda x: 1 if x != 0 else x)
data_pd

,게시글,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,1,0,0,0,0,0,0,0,0
1,모든 게 후회된다,1,0,0,0,0,0,0,0,0
2,임신하고 나서 우울하고 의욕이 없어,1,0,0,0,0,0,0,0,0
3,패배감 너무 심해서 어디 상담이라도 받고 싶어,1,1,0,0,0,0,0,0,0
4,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
59908,그래도 잠못자고 불안한건 여전해요.,0,0,0,0,0,0,0,0,0
59909,불안함에 항상 시달리니까 잠도 못잤어요.,0,0,0,0,0,0,0,0,0
59910,불안하고 초조해서 잠이 안 와.,0,0,0,0,0,0,0,0,0
59911,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,0,0,0,0,0,0,0,0,0


In [ ]:
data_pd_list = data_pd.to_numpy()
data_pd_list

array([[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', 1, 0, ..., 0, 0, 0],
       [' 모든 게 후회된다', 1, 0, ..., 0, 0, 0],
       [' 임신하고 나서 우울하고 의욕이 없어', 1, 0, ..., 0, 0, 0],
       ...,
       ['불안하고 초조해서 잠이 안 와.', 0, 0, ..., 0, 0, 0],
       ['너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.', 0, 0, ..., 0, 0, 0],
       ['불안해서 그런지 요즘 잠도 잘 안 와서 너무 힘들어요.', 0, 0, ..., 0, 0, 0]],
      dtype=object)

In [ ]:
# 훈련, 테스트 데이터셋 분리
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_pd_list, test_size=0.25, random_state=0)

In [ ]:
dataset_train

array([['하루 종일 어디 나가지도 못하니까 눈물이 나는 거야', 1, 0, ..., 0, 0, 0],
       ['나처럼 에타 사진 첨부 안되는 사람 혹시 있니?', 0, 0, ..., 0, 0, 0],
       ['소행론 웹엑스 들어가야해? ㅈㄱㄴ', 0, 0, ..., 0, 0, 0],
       ...,
       ['남친 여친 부모님께 소개시켜주는 경우 흔해?', 0, 0, ..., 0, 0, 0],
       ['중국어 고수분들 중국어 HSK5급 정도 수준가진 학생입니다지금 평소에 공부를 하는 것에 더해서 중국어 글을 좀 읽고 싶은데, 사용하시는 웹사이트나 중국어로 글 읽기 좋은 곳 추천 가능하실까요 부탁드립니다',
        0, 0, ..., 0, 0, 0],
       ['거의 반나절 동안 잠을 잔 것 같아요.', 0, 1, ..., 0, 0, 0]], dtype=object)

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        #tokens_a = self._tokenizer(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')
#[출처] No module named 'kobert' 에러 해결|작성자 yeon



In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#저는 csv 파일을 사용하여 이부분은 dataset_train.values 로 해주었습니다.
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)
#[출처] No module named 'kobert' 에러 해결|작성자 yeon

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 데이터로더 생성
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
data_train[0]

(array([   2, 4937, 4197, 7126, 3223, 1371, 7328, 2086, 7782, 5771, 1537,
        7096, 1375,  862, 6844,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'게시글':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해', '그만 살고 싶어', '나 이제 너무 지쳤어']})
test_df

,게시글
0,우울해서 죽고싶어
1,잠이 안 와
2,요즘 자꾸 살쪄
3,불안하고 심장이 두근거려
4,약 먹은 것처럼 멍해
5,그만 살고 싶어
6,나 이제 너무 지쳤어


In [ ]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'게시글':['요즘 행복해','학교 개강해서 설레고 들떠','우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해', '그만 살고 싶어', '나 이제 너무 지쳤어']}, columns=data_pd.columns)
test_df

,게시글,1,2,3,4,5,6,7,8,9
0,요즘 행복해,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,학교 개강해서 설레고 들떠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,우울해서 죽고싶어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,잠이 안 와,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,요즘 자꾸 살쪄,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,불안하고 심장이 두근거려,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,약 먹은 것처럼 멍해,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,그만 살고 싶어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,나 이제 너무 지쳤어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# KoBERT 모델 이용하여 멀티 라벨 분류

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
#import os

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# 학습

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        outs = out.argmax(1).cpu().numpy()
        labels = label.cpu().numpy()
        f1 = f1_score(outs, labels, average='macro')
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {} F1 score {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1), f1))
    print("epoch {} train acc {} F1 score {}".format(e+1, train_acc / (batch_id+1), f1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        outs = out.argmax(1).cpu().numpy()
        labels = label.cpu().numpy()
        f1 = f1_score(outs, labels, average='macro')
    print("epoch {} test acc {} F1 score {}".format(e+1, test_acc / (batch_id+1), f1))

<ipython-input-33-3448ec8b6b52>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/703 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.557424545288086 train acc 0.015625 F1 score 0.011695906432748537
epoch 1 batch id 101 loss 1.1777493953704834 train acc 0.3307549504950495 F1 score 0.3904761904761905
epoch 1 batch id 201 loss 0.5925868153572083 train acc 0.5128264925373134 F1 score 0.4336283185840708
epoch 1 batch id 301 loss 0.5745924115180969 train acc 0.5768272425249169 F1 score 0.4912559618441971
epoch 1 batch id 401 loss 0.5998256206512451 train acc 0.603841957605985 F1 score 0.5277669902912621
epoch 1 batch id 501 loss 0.6248363852500916 train acc 0.6234718063872255 F1 score 0.6363636363636364
epoch 1 batch id 601 loss 0.4398171901702881 train acc 0.6422628951747088 F1 score 0.5846607669616519
epoch 1 batch id 701 loss 0.39263150095939636 train acc 0.6595042796005706 F1 score 0.8084469571000997
epoch 1 train acc 0.6602062588904695 F1 score 1.0


<ipython-input-33-3448ec8b6b52>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 10 batch id 1 loss 0.02073107846081257 train acc 0.984375 F1 score 0.9803379416282643
epoch 10 batch id 101 loss 0.002013856777921319 train acc 0.9911819306930693 F1 score 1.0
epoch 10 batch id 201 loss 0.008875301107764244 train acc 0.992148631840796 F1 score 1.0
epoch 10 batch id 301 loss 0.0010097112972289324 train acc 0.9925249169435216 F1 score 1.0
epoch 10 batch id 401 loss 0.021176354959607124 train acc 0.9929083541147132 F1 score 0.980991980991981
epoch 10 batch id 501 loss 0.0004721888399217278 train acc 0.9934505988023952 F1 score 1.0
epoch 10 batch id 601 loss 0.0005162618472240865 train acc 0.9934484193011647 F1 score 1.0
epoch 10 batch id 701 loss 0.0015435111708939075 train acc 0.9933131241084165 F1 score 1.0
epoch 10 train acc 0.9933321479374111 F1 score 1.0


<ipython-input-33-3448ec8b6b52>:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/235 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 10 test acc 0.9203900709219858 F1 score 0.6666666666666666


In [ ]:
# KoBERT 모델로 멀티 라벨 결과 예측하기

import numpy as np
import math


def predict(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)

      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)


      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            #logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))

            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)

            df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]
            #nn =(df.shape[1]-1)/ 2
            #df.iloc[i, 1:] = np.tile(temp, (1, nn+1))
            #df.iloc[i, 1:len(temp)+1] = temp

            #logits = logits.detach().cpu().numpy() # 라벨 예측 결과

            #df.iloc[i, 1:] = np.argmax(logits)
            #print(df.iloc[i, 1])

In [ ]:
predict(test_df)

0
['요즘 행복해', '0']


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1, 1, 0, 0, 0, 0, 0, 0, 0]
['학교 개강해서 설레고 들떠', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['우울해서 죽고싶어', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['잠이 안 와', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['요즘 자꾸 살쪄', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['불안하고 심장이 두근거려', '0']
[1, 0, 0, 0, 0, 0, 0, 0, 0]
['약 먹은 것처럼 멍해', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['그만 살고 싶어', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]
['나 이제 너무 지쳤어', '0']
[1, 1, 0, 0, 0, 0, 0, 0, 0]


# 각 라벨별 확률 구하기 (Sigmoid)

In [ ]:
# KoBERT 모델로 예측한 에브리타임 게시물 분류 결과 저장_확률편

import numpy as np

def predict_prb(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))
            print(probability)

'''
            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)
'''

            #df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]

            # logits = logits.detach().cpu().numpy() # 라벨 예측 결과

            # df.iloc[i, 1] = np.argmax(logits)

'\n            # 시그모이드 변환값이 0.5이상이라면 라벨 할당\n            temp = [] # 라벨 저장할 리스트\n            for p in probability:\n              if p > 0.5:\n                temp.append(1)\n              else:\n                temp.append(0)\n            print(temp)\n'

In [ ]:

# 테스트 진행
predict_prb(test_df)

0
['요즘 행복해', '0']
[0.9998423  0.62341577 0.01822204 0.02060077 0.01784469 0.02874685
 0.01699995 0.02546799 0.02583631]
['학교 개강해서 설레고 들떠', '0']
[0.66422486 0.99983096 0.0433444  0.03427358 0.03534203 0.04923901
 0.03279975 0.04647691 0.05095315]
['우울해서 죽고싶어', '0']
[0.9996861  0.9754497  0.00734621 0.00820378 0.0076558  0.01409985
 0.00719116 0.01150739 0.0132484 ]
['잠이 안 와', '0']
[0.99977857 0.9422986  0.00859404 0.00959823 0.00857238 0.01661667
 0.00833321 0.01290993 0.0147823 ]
['요즘 자꾸 살쪄', '0']
[0.99984455 0.6551936  0.01719493 0.01944908 0.01685304 0.02758828
 0.01605072 0.02418028 0.02458772]
['불안하고 심장이 두근거려', '0']
[0.99981576 0.4572705  0.0250763  0.02818943 0.02393492 0.03666322
 0.02324557 0.03328435 0.03400403]
['약 먹은 것처럼 멍해', '0']
[0.9997185  0.96973306 0.00754781 0.00844099 0.0077537  0.01464811
 0.00738092 0.01167194 0.01337961]
['그만 살고 싶어', '0']
[0.99983037 0.5276135  0.02189758 0.02462758 0.02105133 0.03286032
 0.02027837 0.02962725 0.03006439]
['나 이제 너무 지쳤어', '0']
[0.584